# problem

In this competition, the fifth iteration, you will use hierarchical sales data from Walmart, the world’s largest company by revenue, to forecast daily sales for the next 28 days. The data, covers stores in three US States (California, Texas, and Wisconsin) and includes item level, department, product categories, and store details. In addition, it has explanatory variables such as price, promotions, day of the week, and special events. Together, this robust dataset can be used to improve forecasting accuracy.

This competition uses a Weighted Root Mean Squared Scaled Error (RMSSE).

#### Submission

Each row contains an ```id``` that is a concatenation of an ```item_id``` and a ```store_id```, which is either ```validation``` (corresponding to the Public leaderboard), or ```evaluation``` (corresponding to the Private leaderboard). You are predicting 28 forecast days (```F1-F28```) of items sold for each row. For the validation rows, this corresponds to ```d_1914``` - ```d_1941```, and for the evaluation rows, this corresponds to ```d_1942``` - ```d_1969```. (Note: a month before the competition close, the ground truth for the validation rows will be provided.)

Predicting number of items sold for each category for days 1 to 28 in a month. Rolling forecast. 

#### Approach

(1) Get data into a long format, so that we can make time series graphs.
(2) Add the calendar and price datasets to the long format main dataset.
(3) Start most gross and then work into specifics. Look at trends over weeks in product sales.
(4) Group by product category.


#### Random thoughts
(1) These data are hierarchical, so using that hierachy in the model will help to simplify the model. 
(2) Can we combine a model that does weekly estimates with one that estimates for each day of the week? 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
%matplotlib inline
import matplotlib.pyplot as plt

# Quick look at each of the datasets

In [2]:
calendar = pd.read_csv('data/m5-forecasting-accuracy/calendar.csv')

Calendar maps dates onto days, weekday, months, years, special holidays.

In [76]:
calendar.head(5)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,day_code
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0,1
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0,2
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0,3
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0,4
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1,5


In [3]:
calendar['day_code']= calendar['d'].str.lstrip('d_')

In [4]:
calendar['day_code'] = calendar['day_code'].astype(int)

In [5]:
sales_count = pd.read_csv('data/m5-forecasting-accuracy/sales_train_validation.csv')

Sales train validation data tells us about number of products sold each day for each category: hobbies, household, foods. 

In [6]:
sales_count.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [6]:
sales_prices = pd.read_csv('data/m5-forecasting-accuracy/sell_prices.csv')

Sales prices tells us about price of each product for each week.

In [20]:
sales_prices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


# Combining datasets

Let's combine the datasets.

We need to flip from wide to long format for the time series to work. 

Food for thought: let's group by weeks to make this more managable to start with. 

In [7]:
sales_count_long = pd.wide_to_long(sales_count, stubnames='d', i=['item_id', 'store_id'], j='day',
                    sep='_', suffix='\w+')

In [11]:
sales_count_long = sales_count_long.reset_index()

In [12]:
sales_count_long.head()

,item_id,store_id,day,dept_id,cat_id,id,state_id,d
0,HOBBIES_1_001,CA_1,1,HOBBIES_1,HOBBIES,HOBBIES_1_001_CA_1_validation,CA,0
1,HOBBIES_1_001,CA_1,2,HOBBIES_1,HOBBIES,HOBBIES_1_001_CA_1_validation,CA,0
2,HOBBIES_1_001,CA_1,3,HOBBIES_1,HOBBIES,HOBBIES_1_001_CA_1_validation,CA,0
3,HOBBIES_1_001,CA_1,4,HOBBIES_1,HOBBIES,HOBBIES_1_001_CA_1_validation,CA,0
4,HOBBIES_1_001,CA_1,5,HOBBIES_1,HOBBIES,HOBBIES_1_001_CA_1_validation,CA,0


In [83]:
sales_count_long.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58327370 entries, 0 to 58327369
Data columns (total 6 columns):
level_0     int64
index       int64
item_id     object
store_id    object
day         int64
d           int64
dtypes: int64(4), object(2)
memory usage: 2.6+ GB


Now we merge the calendar info onto the sales_count_long dataset. 

In [13]:
sales_count_long_cal = pd.merge(sales_count_long, calendar, left_on = 'day', right_on = 'day_code')

In [14]:
sales_count_long_cal.head()

,item_id,store_id,day,dept_id,cat_id,id,state_id,d_x,date,wm_yr_wk,...,year,d_y,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,day_code
0,HOBBIES_1_001,CA_1,1,HOBBIES_1,HOBBIES,HOBBIES_1_001_CA_1_validation,CA,0,2011-01-29,11101,...,2011,d_1,NaN,NaN,NaN,NaN,0,0,0,1
1,HOBBIES_1_002,CA_1,1,HOBBIES_1,HOBBIES,HOBBIES_1_002_CA_1_validation,CA,0,2011-01-29,11101,...,2011,d_1,NaN,NaN,NaN,NaN,0,0,0,1
2,HOBBIES_1_003,CA_1,1,HOBBIES_1,HOBBIES,HOBBIES_1_003_CA_1_validation,CA,0,2011-01-29,11101,...,2011,d_1,NaN,NaN,NaN,NaN,0,0,0,1
3,HOBBIES_1_004,CA_1,1,HOBBIES_1,HOBBIES,HOBBIES_1_004_CA_1_validation,CA,0,2011-01-29,11101,...,2011,d_1,NaN,NaN,NaN,NaN,0,0,0,1
4,HOBBIES_1_005,CA_1,1,HOBBIES_1,HOBBIES,HOBBIES_1_005_CA_1_validation,CA,0,2011-01-29,11101,...,2011,d_1,NaN,NaN,NaN,NaN,0,0,0,1


In [15]:
sales_count_long_cal_price = pd.merge(sales_count_long_cal, sales_prices, on = ['item_id', 'store_id', 'wm_yr_wk'])

In [16]:
sales_count_long_cal_price.head()

,item_id,store_id,day,dept_id,cat_id,id,state_id,d_x,date,wm_yr_wk,...,d_y,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,day_code,sell_price
0,HOBBIES_1_008,CA_1,1,HOBBIES_1,HOBBIES,HOBBIES_1_008_CA_1_validation,CA,12,2011-01-29,11101,...,d_1,NaN,NaN,NaN,NaN,0,0,0,1,0.46
1,HOBBIES_1_008,CA_1,2,HOBBIES_1,HOBBIES,HOBBIES_1_008_CA_1_validation,CA,15,2011-01-30,11101,...,d_2,NaN,NaN,NaN,NaN,0,0,0,2,0.46
2,HOBBIES_1_008,CA_1,3,HOBBIES_1,HOBBIES,HOBBIES_1_008_CA_1_validation,CA,0,2011-01-31,11101,...,d_3,NaN,NaN,NaN,NaN,0,0,0,3,0.46
3,HOBBIES_1_008,CA_1,4,HOBBIES_1,HOBBIES,HOBBIES_1_008_CA_1_validation,CA,0,2011-02-01,11101,...,d_4,NaN,NaN,NaN,NaN,1,1,0,4,0.46
4,HOBBIES_1_008,CA_1,5,HOBBIES_1,HOBBIES,HOBBIES_1_008_CA_1_validation,CA,0,2011-02-02,11101,...,d_5,NaN,NaN,NaN,NaN,1,0,1,5,0.46


# Feature Engineering

Converting date into weeks

In [17]:
sales_count_long_cal_price['date'] = pd.to_datetime(sales_count_long_cal_price['date'])

In [18]:
sales_count_long_cal_price['week'] = sales_count_long_cal_price.date.dt.floor('D')- pd.to_timedelta(sales_count_long_cal_price['date'].dt.dayofweek, unit='d')

Let's look at the event_name variables. These could be dummy coded for each event (like Mike has done already). 

In [37]:
sales_count_long_cal_price['event_name_1'].unique()

array([nan, 'SuperBowl', 'ValentinesDay', 'PresidentsDay', 'LentStart',
       'LentWeek2', 'StPatricksDay', 'Purim End', 'OrthodoxEaster',
       'Pesach End', 'Cinco De Mayo', "Mother's day", 'MemorialDay',
       'NBAFinalsStart', 'NBAFinalsEnd', "Father's day",
       'IndependenceDay', 'Ramadan starts', 'Eid al-Fitr', 'LaborDay',
       'ColumbusDay', 'Halloween', 'EidAlAdha', 'VeteransDay',
       'Thanksgiving', 'Christmas', 'Chanukah End', 'NewYear',
       'OrthodoxChristmas', 'MartinLutherKingDay', 'Easter'], dtype=object)

In [23]:
sales_count_long_cal_price['event_type_1'].unique()

array([nan, 'Sporting', 'Cultural', 'National', 'Religious'], dtype=object)

In [21]:
sales_count_long_cal_price['event_name_2'].unique()

array([nan, 'Easter', 'Cinco De Mayo', 'OrthodoxEaster', "Father's day"],
      dtype=object)

In [24]:
sales_count_long_cal_price['event_type_2'].unique()

array([nan, 'Cultural', 'Religious'], dtype=object)

There are 29 events and 4 categories of event. These all become dummy variables.
Right now event 1 and event 2 have their own set of dummy variables. Loop back to this later to condense the events into one set of dummy variables. Not sure how to do this... if statement iterating over the events? 

In [54]:
sales_count_long_cal_price_event_dummies = pd.get_dummies(sales_count_long_cal_price, columns=['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2'])

In [55]:
sales_count_long_cal_price_event_dummies.head()

,item_id,store_id,day,dept_id,cat_id,id,state_id,d_x,date,wm_yr_wk,...,event_type_1_Cultural,event_type_1_National,event_type_1_Religious,event_type_1_Sporting,event_name_2_Cinco De Mayo,event_name_2_Easter,event_name_2_Father's day,event_name_2_OrthodoxEaster,event_type_2_Cultural,event_type_2_Religious
0,HOBBIES_1_008,CA_1,1,HOBBIES_1,HOBBIES,HOBBIES_1_008_CA_1_validation,CA,12,2011-01-29,11101,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_008,CA_1,2,HOBBIES_1,HOBBIES,HOBBIES_1_008_CA_1_validation,CA,15,2011-01-30,11101,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_008,CA_1,3,HOBBIES_1,HOBBIES,HOBBIES_1_008_CA_1_validation,CA,0,2011-01-31,11101,...,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_008,CA_1,4,HOBBIES_1,HOBBIES,HOBBIES_1_008_CA_1_validation,CA,0,2011-02-01,11101,...,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_008,CA_1,5,HOBBIES_1,HOBBIES,HOBBIES_1_008_CA_1_validation,CA,0,2011-02-02,11101,...,0,0,0,0,0,0,0,0,0,0


Saving this dataset to file. Yikes it is huge! 

In [56]:
sales_count_long_cal_price_event_dummies.to_csv('combined_long_data.csv')